# Case: Analytics Engineer  SR - Antonio Ferrazzoli

In [2]:
pip install apache-airflow

Note: you may need to restart the kernel to use updated packages.


In [10]:
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.contrib.operators.bigquery_operator import BigQueryOperator

default_args = {
    'owner': 'Antonio Ferrazzoli',
    'depends_on_past': False,
    'start_date': datetime(2024, 2, 15),  
    'email_on_failure': True,
    'email': ['antonioferrazzoli16@gmail.com'],
    'email_on_retry': True,
    'retries': 3,  
    'retry_delay': timedelta(minutes=5) 
}

dag = DAG(
    'processamento_com_airflow',
    default_args=default_args,
    description='executar quinta feita 7 da mnha',
    schedule_interval='0 7 * * 4' 
)

q1 = BigQueryOperator(
    task_id='executar_query_q1',
    sql='Q1.sql',
    use_legacy_sql=False,
    params={'date_max': '{{ execution_date }}',
            'date_min': '{{ execution_date - macros.timedelta(days=3) }}'},
    dag=dag
)

py = BashOperator(
    task_id='caixa-preta',
    bash_command='Projetos Python\Estatísticas com PY\Case Carrefour\Case Carrefour notebook - Data Science.py',
    dag=dag
)


q1 >> py


AirflowConfigException: Cannot use relative path: `sqlite:///C:\Users\Antonio Ferrazzoli/airflow/airflow.db` to connect to sqlite. Please use absolute path such as `sqlite:////tmp/airflow.db`.

Item b)

Para este item será adicionado uma terceita task na dag para pode gravar o resultado do arquivo py no bq, segue código:

In [ ]:
carregar_bigquery_task = BigQueryOperator(
    task_id='carregar_bigquery',
    sql='''
    LOAD DATA
        PROJETO.DATASET.TABLE
    FROM 'Projetos Python\Estatísticas com PY\Case Carrefour\Case Carrefour notebook - Data Science.py'
    FORMAT CSV
    ''',  
    destination_dataset_table='your_project.your_dataset.your_table',  # Substitua pelos detalhes da sua tabela no BigQuery
    write_disposition='WRITE_TRUNCATE',  # Ou WRITE_APPEND, WRITE_EMPTY
    dag=dag
)